In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import pywt.data
import sys
import cv2

if '..' not in sys.path:
    sys.path.append('..')

In proposed MDLE-IWT method, 
1. IWT is first appliedto the cover image to obtain four wavelet coefficient sub-bands asembedding regions.
1. The embedding sub-bands order in the algorithm is HL, LH, HH, and LL. 
1. Next, the embedding sub-band is divided into 3×3 non-overlapping blocks
1. MDLE scheme is applied to each block


In [ ]:
from itertools import chain, repeat, cycle
from stego import codec
from stego.message import message_to_dec, dec_to_message
from stego.transform.dwt import Iwt
from stego.image_coder import StegoCoder
import unireedsolomon as rs

## Encoding

In [ ]:
from stego import correction_codes, image_coder

secret_message = "Functions creating iterators for efficient looping"
original = pywt.data.camera()
stego_coder = StegoCoder(Iwt('haar', level=3, scale=1), len(secret_message))

In [ ]:
stego_img = stego_coder.encode(original, secret_message)

## Stego Image

In [ ]:
plt.imshow(stego_img, interpolation="nearest", cmap=plt.cm.gray)
cv2.imwrite("comp.png", stego_img)
img = cv2.imread("comp.png")
stego_from_file, g, b = cv2.split(img)
print(f"{np.allclose(stego_img, stego_from_file)=}")
stego_img = stego_from_file
plt.imshow(stego_img, interpolation="nearest", cmap=plt.cm.gray)
stego_img.shape


## Extracting

In [ ]:
message = stego_coder.decode(stego_img)
message

## Przeanalizowanie obrazka testowego oraz wyników przekształcenia na poziomie 3 pod kątem różnicy pomiędzy pikselami w obszarze 3x3

In [ ]:
from stego.transform.blocking import CropBlocker

original_blocking = CropBlocker(original)
stego_blocking = CropBlocker(stego_img)

In [ ]:
original_blocks = original_blocking.divide(block_size=3)
stego_blocks = stego_blocking.divide(block_size=3)

In [ ]:
for j, row in enumerate(original_blocks):
    for k, block in enumerate(row):
        s = np.std(block)
        original_blocks[j][k] = (block * 0) + s

orginal_std = original_blocking.stack(original_blocks)


In [ ]:
for j, row in enumerate(stego_blocks):
    for k, block in enumerate(row):
        s = np.std(block)
        stego_blocks[j][k] = (block * 0) + s

stego_std = stego_blocking.stack(stego_blocks)

In [ ]:
fig = plt.figure(figsize=(15, 5))
for i, a in enumerate([orginal_std, stego_std]):
    ax = fig.add_subplot(1, 4, i + 1)
    ax.imshow(a, cmap=plt.cm.gray)
    ax = fig.add_subplot(1, 4, i + 3)
    ax.hist(a)

fig.tight_layout()
plt.show()